<a href="https://colab.research.google.com/github/harshi1707/FinalProject/blob/main/Reinforcement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install -q streamlit pyngrok pandas numpy scikit-learn tensorflow torch gymnasium stable-baselines3

# Import libraries
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
import torch
import torch.nn as nn
import torch.optim as optim
from gymnasium import Env
from gymnasium.spaces import Box
from stable_baselines3 import PPO, SAC
from pyngrok import ngrok
import os
import matplotlib.pyplot as plt
from datetime import datetime, timezone

# Define a simple Gymnasium environment for crypto token optimization
class CryptoEnv(Env):
    def __init__(self):
        super(CryptoEnv, self).__init__()
        self.action_space = Box(low=-1, high=1, shape=(1,), dtype=np.float32)  # Adjust token amount
        self.observation_space = Box(low=0, high=1e6, shape=(3,), dtype=np.float32)  # Amount, rating, time
        self.state = None
        self.step_count = 0
        self.max_steps = 100

    def reset(self, seed=None):
        self.state = np.array([1000.0, 100.0, 0.0])  # Initial: amount, rating, time
        self.step_count = 0
        return self.state, {}

    def step(self, action):
        self.step_count += 1
        amount, rating, time = self.state
        action = action[0]  # Adjust token amount
        amount += action * 200  # Scale action
        rating += np.random.normal(0, 2)  # Random rating fluctuation (simulating gas/trust)
        time += 1
        self.state = np.array([max(0, amount), max(0, rating), time])
        reward = amount * (rating / 10) if amount > 0 else -abs(amount)  # Optimize for value, reward high trust
        done = self.step_count >= self.max_steps or amount < 0
        return self.state, reward, done, False, {}

# Simple A3C Actor-Critic Network
class ActorCritic(nn.Module):
    def __init__(self, input_dim, action_dim):
        super(ActorCritic, self).__init__()
        self.fc = nn.Linear(input_dim, 64)
        self.actor = nn.Linear(64, action_dim)
        self.critic = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc(x))
        action = torch.tanh(self.actor(x))  # Bounded action
        value = self.critic(x)
        return action, value

# A3C Training (Simplified)
def train_a3c(env, model, optimizer, episodes=10):
    for _ in range(episodes):
        state, _ = env.reset()
        done = False
        while not done:
            state_tensor = torch.FloatTensor(state).unsqueeze(0)
            action, value = model(state_tensor)
            # Keep action as tensor, convert to numpy only for env.step
            next_state, reward, done, _, _ = env.step(action.detach().cpu().numpy()[0])
            next_state_tensor = torch.FloatTensor(next_state).unsqueeze(0)
            _, next_value = model(next_state_tensor)
            td_error = reward + (1 - done) * 0.99 * next_value - value
            # Ensure action is a tensor for log
            loss = -torch.log(action[0]) * td_error + td_error.pow(2)  # Simplified loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            state = next_state
    return model

# Autoencoder for anomaly detection
def build_autoencoder(input_dim):
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(32, activation='relu')(input_layer)
    encoded = Dense(16, activation='relu')(encoded)
    decoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

# Simple Transformer for time-series prediction
class SimpleTransformer(nn.Module):
    def __init__(self, input_dim, model_dim=32):
        super(SimpleTransformer, self).__init__()
        self.input_fc = nn.Linear(input_dim, model_dim)
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=model_dim, nhead=4, dim_feedforward=128), num_layers=2)
        self.fc = nn.Linear(model_dim, 1)

    def forward(self, x):
        x = self.input_fc(x)
        x = self.encoder(x.unsqueeze(0))
        return self.fc(x.squeeze(0))

# Write Streamlit app to file
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
import torch
import torch.nn as nn
import torch.optim as optim
from gymnasium import Env
from gymnasium.spaces import Box
from stable_baselines3 import PPO, SAC
import matplotlib.pyplot as plt
from datetime import datetime, timezone

# Define CryptoEnv
class CryptoEnv(Env):
    def __init__(self):
        super(CryptoEnv, self).__init__()
        self.action_space = Box(low=-1, high=1, shape=(1,), dtype=np.float32)
        self.observation_space = Box(low=0, high=1e6, shape=(3,), dtype=np.float32)
        self.state = None
        self.step_count = 0
        self.max_steps = 100

    def reset(self, seed=None):
        self.state = np.array([1000.0, 100.0, 0.0])
        self.step_count = 0
        return self.state, {}

    def step(self, action):
        self.step_count += 1
        amount, rating, time = self.state
        action = action[0]
        amount += action * 200
        rating += np.random.normal(0, 2)
        time += 1
        self.state = np.array([max(0, amount), max(0, rating), time])
        reward = amount * (rating / 10) if amount > 0 else -abs(amount)
        done = self.step_count >= self.max_steps or amount < 0
        return self.state, reward, done, False, {}

# Simple A3C Actor-Critic Network
class ActorCritic(nn.Module):
    def __init__(self, input_dim, action_dim):
        super(ActorCritic, self).__init__()
        self.fc = nn.Linear(input_dim, 64)
        self.actor = nn.Linear(64, action_dim)
        self.critic = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc(x))
        action = torch.tanh(self.actor(x))  # Bounded action
        value = self.critic(x)
        return action, value

# A3C Training (Simplified)
def train_a3c(env, model, optimizer, episodes=10):
    for _ in range(episodes):
        state, _ = env.reset()
        done = False
        while not done:
            state_tensor = torch.FloatTensor(state).unsqueeze(0)
            action, value = model(state_tensor)
            # Keep action as tensor, convert to numpy only for env.step
            next_state, reward, done, _, _ = env.step(action.detach().cpu().numpy()[0])
            next_state_tensor = torch.FloatTensor(next_state).unsqueeze(0)
            _, next_value = model(next_state_tensor)
            td_error = reward + (1 - done) * 0.99 * next_value - value
            # Ensure action is a tensor for log
            loss = -torch.log(action[0]) * td_error + td_error.pow(2)  # Simplified loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            state = next_state
    return model

# Autoencoder for anomaly detection
def build_autoencoder(input_dim):
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(32, activation='relu')(input_layer)
    encoded = Dense(16, activation='relu')(encoded)
    decoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

# Simple Transformer for time-series prediction
class SimpleTransformer(nn.Module):
    def __init__(self, input_dim, model_dim=32):
        super(SimpleTransformer, self).__init__()
        self.input_fc = nn.Linear(input_dim, model_dim)
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=model_dim, nhead=4, dim_feedforward=128), num_layers=2)
        self.fc = nn.Linear(model_dim, 1)

    def forward(self, x):
        x = self.input_fc(x)
        x = self.encoder(x.unsqueeze(0))
        return self.fc(x.squeeze(0))

# Streamlit dashboard
st.set_page_config(page_title="Crypto Token Optimization", layout="wide")
st.title("Real-Time Crypto Token Optimization")

# Load Bitcoin OTC dataset from public URL
try:
    df = pd.read_csv('https://snap.stanford.edu/data/soc-sign-bitcoinotc.csv.gz', compression='gzip', header=None, names=['source', 'target', 'rating', 'time'])
    df['time'] = pd.to_datetime(df['time'], unit='s')
    X = df[['rating', 'time']].copy()
    X['time'] = (X['time'] - X['time'].min()).dt.total_seconds()  # Convert time to seconds
    X = X.fillna(0)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
except Exception as e:
    st.error(f"Dataset load error: {e}. Using synthetic data.")
    X_scaled = np.random.rand(1000, 2)
    df = pd.DataFrame({'rating': X_scaled[:, 0], 'time': X_scaled[:, 1]})

# Anomaly detection - Autoencoder
input_dim = X_scaled.shape[1]
autoencoder = build_autoencoder(input_dim)
autoencoder.fit(X_scaled, X_scaled, epochs=5, batch_size=128, verbose=0)
recon = autoencoder.predict(X_scaled)
mse = np.mean(np.power(X_scaled - recon, 2), axis=1)
threshold = np.percentile(mse, 95)
autoencoder_anomalies = mse > threshold

# Isolation Forest
iso_forest = IsolationForest(contamination=0.01, random_state=42)
iso_anomalies = iso_forest.fit_predict(X_scaled) == -1

# Hybrid Model (Autoencoder + Isolation Forest)
hybrid_anomalies = np.logical_or(autoencoder_anomalies, iso_anomalies)

# Transformer prediction
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transformer = SimpleTransformer(input_dim=X_scaled.shape[1]).to(device)
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.001)
for _ in range(5):
    inputs = torch.tensor(X_scaled, dtype=torch.float32).to(device)
    targets = inputs[:, 0].unsqueeze(1)
    optimizer.zero_grad()
    outputs = transformer(inputs)
    loss = nn.MSELoss()(outputs, targets)
    loss.backward()
    optimizer.step()

# RL models
env = CryptoEnv()
ppo_model = PPO("MlpPolicy", env, verbose=0).learn(total_timesteps=1000)
sac_model = SAC("MlpPolicy", env, verbose=0).learn(total_timesteps=1000)
a3c_model = ActorCritic(input_dim=3, action_dim=1).to(device)
a3c_optimizer = optim.Adam(a3c_model.parameters(), lr=0.001)
a3c_model = train_a3c(env, a3c_model, a3c_optimizer)

# Simulate RL
def simulate_rl(model, env, n_steps=100):
    obs, _ = env.reset()
    rewards = []
    for _ in range(n_steps):
        if isinstance(model, ActorCritic):
            state_tensor = torch.FloatTensor(obs).unsqueeze(0).to(device)
            action, _ = model(state_tensor)
            action = action.detach().cpu().numpy()[0]
        else:
            action, _ = model.predict(obs)
        obs, reward, done, _, _ = env.step(action)
        rewards.append(reward)
        if done:
            obs, _ = env.reset()
    return rewards

ppo_rewards = simulate_rl(ppo_model, env)
sac_rewards = simulate_rl(sac_model, env)
a3c_rewards = simulate_rl(a3c_model, env)

# Dashboard
st.header("Anomaly Detection")
col1, col2, col3 = st.columns(3)
with col1:
    st.subheader("Autoencoder")
    st.write(f"Anomalies: {sum(autoencoder_anomalies)}")
    fig, ax = plt.subplots()
    ax.hist(mse, bins=50, log=True)
    ax.axvline(threshold, color='r', linestyle='--')
    st.pyplot(fig)
with col2:
    st.subheader("Isolation Forest")
    st.write(f"Anomalies: {sum(iso_anomalies)}")
    fig, ax = plt.subplots()
    ax.scatter(X_scaled[:, 0], X_scaled[:, 1], c=iso_anomalies, cmap='coolwarm')
    st.pyplot(fig)
with col3:
    st.subheader("Hybrid (Auto + Iso)")
    st.write(f"Anomalies: {np.sum(hybrid_anomalies)}")
    fig, ax = plt.subplots()
    ax.scatter(X_scaled[:, 0], X_scaled[:, 1], c=hybrid_anomalies, cmap='coolwarm')
    st.pyplot(fig)

st.header("RL Optimization Performance")
fig, ax = plt.subplots()
ax.plot(np.cumsum(ppo_rewards), label='PPO')
ax.plot(np.cumsum(sac_rewards), label='SAC')
ax.plot(np.cumsum(a3c_rewards), label='A3C')
ax.set_title("Cumulative Rewards (Token Optimization)")
ax.legend()
st.pyplot(fig)

st.header("Transformer Prediction")
with torch.no_grad():
    pred = transformer(torch.tensor(X_scaled[:10], dtype=torch.float32).to(device)).cpu().numpy()
fig, ax = plt.subplots()
ax.plot(pred, label='Predicted Rating')
ax.plot(X_scaled[:10, 0], label='Actual Rating')
ax.legend()
st.pyplot(fig)
""")

# Run Streamlit with ngrok
!ngrok authtoken 31r5s9qEOLsZND89IKnKitqkKKp_48taEqEuyUcMZ7r5RhVqj
!nohup streamlit run app.py --server.port 8501 &
public_url = ngrok.connect(8501)
print(f"Streamlit app running at: {public_url}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
nohup: appending output to 'nohup.out'
Streamlit app running at: NgrokTunnel: "https://0c2492ad11fe.ngrok-free.app" -> "http://localhost:8501"
